# Quantitative Trading System - Main Workflow

This notebook is the main interface for the quantitative trading system.

## Workflow Overview
1. **Setup & Initialization** - Import libraries and configure environment
2. **Configuration** - Load settings from `.env` file
3. **Connection** - Establish connection to IB Gateway/TWS
4. **Account Information** - View account details and positions
5. **Trading Operations** - Execute strategies and manage positions

---

## 1. Setup & Initialization

Import all necessary libraries and configure the Python environment.

In [1]:
# Enable auto-reload for module changes
# This allows you to edit .py files without restarting the kernel
%load_ext autoreload
%autoreload 2

# Standard library imports
import sys
import os
from pathlib import Path
from datetime import datetime
import logging

# Data analysis libraries
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# IB API
from ib_insync import IB, Stock, util

# Configure path to import our custom modules
# Get the parent directory (project root)
notebook_dir = Path.cwd()
if notebook_dir.name == 'src':
    project_root = notebook_dir.parent
    src_path = notebook_dir
else:
    project_root = notebook_dir
    src_path = notebook_dir / 'src'

# Add src to Python path if not already there
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import our custom classes
from config.config import Config
from connection.ib_connection import IBConnection

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Set pandas display options
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

# Set matplotlib style
plt.style.use('seaborn-v0_8-darkgrid')

print("✅ All libraries imported successfully!")
print(f"📁 Project Root: {project_root}")
print(f"📁 Source Path: {src_path}")
print(f"🐍 Python Version: {sys.version.split()[0]}")
print("\n🔄 Auto-reload enabled - module changes will be reflected automatically!")

✅ All libraries imported successfully!
📁 Project Root: /workspaces/Quant
📁 Source Path: /workspaces/Quant/src
🐍 Python Version: 3.12.1

🔄 Auto-reload enabled - module changes will be reflected automatically!


## 2. Configuration

Load configuration from the `.env` file.

In [2]:
# Load configuration
config = Config()

print("\n" + "="*60)
print("  CONFIGURATION LOADED")
print("="*60)
print(f"\nHost:        {config.ib_host}")
print(f"Port:        {config.ib_port}")
print(f"Client ID:   {config.ib_client_id}")
print(f"Mode:        {config.trading_mode.upper()}")
print(f"Username:    {'***' if config.ib_username else 'NOT SET'}")
print(f"Account ID:  {'***' if config.ib_account_id else 'NOT SET'}")

if config.is_paper_trading:
    print("\n⚠️  PAPER TRADING MODE - Safe for testing")
else:
    print("\n⚠️  LIVE TRADING MODE - Real money at risk!")

print("\n✅ Configuration loaded successfully!")

2026-02-12 02:42:52 - config.config - INFO - Loaded configuration from /workspaces/Quant/.env



  CONFIGURATION LOADED

Host:        127.0.0.1
Port:        4002
Client ID:   1
Mode:        PAPER
Username:    ***
Account ID:  ***

⚠️  PAPER TRADING MODE - Safe for testing

✅ Configuration loaded successfully!


## 3. Initialize Connection Object

Create the IB Gateway connection manager.

In [3]:
# Create IB connection object
ib_conn = IBConnection(config)

print("\n" + "="*60)
print("  CONNECTION OBJECT CREATED")
print("="*60)
print(f"\n{ib_conn}")
print("\n✅ IBConnection object initialized!")
print("\n📌 Next: Run the connection cell to establish connection to IB Gateway")


  CONNECTION OBJECT CREATED

IBConnection(status=Disconnected, host=127.0.0.1:4002, mode=paper)

✅ IBConnection object initialized!

📌 Next: Run the connection cell to establish connection to IB Gateway


## 4. Establish Connection to IB Gateway/TWS

**PREREQUISITES:**
- IB Gateway or TWS must be running
- You must be logged in
- API access must be enabled in settings

This is the **connection page** - the main entry point for connecting to Interactive Brokers.

In [4]:
# Connect to IB Gateway/TWS
print("\n" + "="*60)
print("  CONNECTING TO IB GATEWAY/TWS")
print("="*60)

print(f"\n🔌 Attempting connection...")
print(f"   Host: {config.ib_host}")
print(f"   Port: {config.ib_port}")
print(f"   Client ID: {config.ib_client_id}")

# Attempt connection
success = ib_conn.connect(timeout=10)

if success:
    print("\n" + "="*60)
    print("  ✅ CONNECTION SUCCESSFUL!")
    print("="*60)
    print(f"\nStatus: {ib_conn.is_connected}")
    print(f"Mode: {config.trading_mode.upper()}")
    print("\n🎉 You are now connected to Interactive Brokers!")
    print("📊 You can now proceed to view account info and execute trades.")
else:
    print("\n" + "="*60)
    print("  ❌ CONNECTION FAILED")
    print("="*60)
    print("\n🔧 Troubleshooting:")
    print("   1. Ensure IB Gateway/TWS is running")
    print("   2. Check that you are logged in")
    print("   3. Verify API settings:")
    print("      - Go to Configure → Settings → API → Settings")
    print("      - Enable 'Enable ActiveX and Socket Clients'")
    print("      - Check 'Read-Only API' for paper trading")
    print(f"   4. Verify port number matches .env: {config.ib_port}")
    print("      - TWS Paper: 7497")
    print("      - TWS Live: 7496")
    print("      - IB Gateway Paper: 4002")
    print("      - IB Gateway Live: 4001")
    print("   5. Check firewall settings")

2026-02-12 02:42:53 - connection.ib_connection - INFO - Connecting to IB Gateway at 127.0.0.1:4002 (Client ID: 1, Mode: paper)
2026-02-12 02:42:53 - connection.ib_connection - ERROR - Connection error: This event loop is already running



  CONNECTING TO IB GATEWAY/TWS

🔌 Attempting connection...
   Host: 127.0.0.1
   Port: 4002
   Client ID: 1

  ❌ CONNECTION FAILED

🔧 Troubleshooting:
   1. Ensure IB Gateway/TWS is running
   2. Check that you are logged in
   3. Verify API settings:
      - Go to Configure → Settings → API → Settings
      - Enable 'Enable ActiveX and Socket Clients'
      - Check 'Read-Only API' for paper trading
   4. Verify port number matches .env: 4002
      - TWS Paper: 7497
      - TWS Live: 7496
      - IB Gateway Paper: 4002
      - IB Gateway Live: 4001
   5. Check firewall settings


2026-02-12 02:42:53 - ib_insync.client - INFO - Connecting to 127.0.0.1:4002 with clientId 1...
2026-02-12 02:42:53 - ib_insync.client - INFO - Disconnecting
2026-02-12 02:42:53 - ib_insync.client - ERROR - API connection failed: ConnectionRefusedError(111, "Connect call failed ('127.0.0.1', 4002)")
2026-02-12 02:42:53 - ib_insync.client - ERROR - Make sure API port on TWS/IBG is open


## 5. Test Connection & Get Server Info

Verify connection and retrieve server information.

In [ ]:
# Test connection and get details
if ib_conn.is_connected:
    test_result = ib_conn.test_connection()
    
    print("\n" + "="*60)
    print("  CONNECTION TEST RESULTS")
    print("="*60)
    print(f"\nConnected:        {test_result['connected']}")
    print(f"Server Version:   {test_result['server_version']}")
    print(f"Connection Time:  {test_result['connection_time']}")
    print(f"Managed Accounts: {test_result.get('accounts', [])}")
    print(f"Position Count:   {test_result['positions_count']}")
    print("\n✅ Connection test completed!")
else:
    print("\n⚠️  Not connected. Please run the connection cell above first.")

## 6. Account Summary

View key account metrics and balances.

In [ ]:
# Get account values
if ib_conn.is_connected:
    account_values = ib_conn.get_account_values()
    
    print("\n" + "="*60)
    print("  ACCOUNT SUMMARY")
    print("="*60)
    
    # Define key metrics to display
    key_metrics = [
        ('NetLiquidation', 'Net Liquidation Value'),
        ('TotalCashValue', 'Total Cash'),
        ('GrossPositionValue', 'Gross Position Value'),
        ('BuyingPower', 'Buying Power'),
        ('AvailableFunds', 'Available Funds'),
        ('MaintMarginReq', 'Maintenance Margin'),
    ]
    
    print(f"\n{'Metric':<30} {'Value':>20}")
    print("-" * 52)
    
    for metric_key, metric_name in key_metrics:
        # Try USD currency first
        value = account_values.get(f"{metric_key}_USD")
        if value is None:
            value = account_values.get(metric_key, 'N/A')
        
        # Format as currency if numeric
        if value != 'N/A':
            try:
                value_float = float(value)
                value_str = f"${value_float:,.2f}"
            except:
                value_str = str(value)
        else:
            value_str = value
        
        print(f"{metric_name:<30} {value_str:>20}")
    
    print("\n✅ Account summary retrieved!")
else:
    print("\n⚠️  Not connected. Please run the connection cell first.")

## 7. Current Positions

View all open positions in the account.

In [ ]:
# Get current positions
if ib_conn.is_connected:
    positions = ib_conn.get_positions()
    
    print("\n" + "="*60)
    print("  CURRENT POSITIONS")
    print("="*60)
    
    if positions:
        print(f"\nTotal Positions: {len(positions)}\n")
        print(f"{'Symbol':<10} {'Quantity':>10} {'Avg Cost':>12} {'Account':<15}")
        print("-" * 50)
        
        for pos in positions:
            symbol = pos.contract.symbol
            quantity = pos.position
            avg_cost = pos.avgCost
            account = pos.account
            
            print(f"{symbol:<10} {quantity:>10.2f} ${avg_cost:>10.2f} {account:<15}")
        
        print("\n✅ Positions retrieved!")
    else:
        print("\n📭 No open positions")
else:
    print("\n⚠️  Not connected. Please run the connection cell first.")

## 8. Portfolio Details

View detailed portfolio information including market values and P&L.

In [ ]:
# Get portfolio items
if ib_conn.is_connected:
    portfolio = ib_conn.get_portfolio_items()
    
    print("\n" + "="*60)
    print("  PORTFOLIO DETAILS")
    print("="*60)
    
    if portfolio:
        print(f"\nTotal Portfolio Items: {len(portfolio)}\n")
        print(f"{'Symbol':<10} {'Quantity':>10} {'Mkt Value':>15} {'Unrealized PnL':>18}")
        print("-" * 55)
        
        total_value = 0
        total_pnl = 0
        
        for item in portfolio:
            symbol = item.contract.symbol
            quantity = item.position
            market_value = item.marketValue
            unrealized_pnl = item.unrealizedPNL
            
            total_value += market_value
            total_pnl += unrealized_pnl
            
            pnl_color = '🟢' if unrealized_pnl >= 0 else '🔴'
            
            print(f"{symbol:<10} {quantity:>10.2f} ${market_value:>13,.2f} {pnl_color} ${unrealized_pnl:>13,.2f}")
        
        print("-" * 55)
        print(f"{'TOTAL':<10} {'':<10} ${total_value:>13,.2f}    ${total_pnl:>13,.2f}")
        
        print("\n✅ Portfolio details retrieved!")
    else:
        print("\n📭 No portfolio items")
else:
    print("\n⚠️  Not connected. Please run the connection cell first.")

## 9. Disconnect

Always disconnect when done to free up resources.

In [ ]:
# Disconnect from IB Gateway
if ib_conn.is_connected:
    print("\n🔌 Disconnecting from IB Gateway/TWS...")
    ib_conn.disconnect()
    print(f"\n✅ Disconnected successfully!")
    print(f"Connection Status: {ib_conn.is_connected}")
else:
    print("\n⚠️  Already disconnected.")

---

## Next Steps

Now that you have a working connection to IB Gateway, you can:

1. **Explore Market Data**
   - Retrieve real-time quotes
   - Fetch historical data
   - Stream market data

2. **Develop Trading Strategies**
   - Create strategy classes in `src/strategies/`
   - Backtest on historical data
   - Test in paper trading

3. **Build Risk Management**
   - Position sizing
   - Stop losses
   - Portfolio limits

4. **Execute Trades**
   - Place orders
   - Monitor fills
   - Track performance

## Resources

- **Config Class**: `src/config/config.py`
- **Connection Class**: `src/connection/ib_connection.py`
- **IB Setup Guide**: `docs/IB_SETUP.md`
- **Test Script**: `python src/test_connection.py`